# Reddit news

In [2]:
import pandas as pd
import sqlite3

In [22]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
kolonas = ['date']+['top'+str(i) for i in range(1,31)]
print(kolonas)

In [ ]:
df = pd.DataFrame(columns = kolonas)

In [ ]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()

In [ ]:
sql = ("Select date(moment) as date, title from top30 where date = '2021-05-05'")
z = pd.read_sql_query(sql,connection)

In [ ]:
for d in z.date.unique():
    atbilde =[]
    atbilde.append(d)
    #print(atbilde)
    for title in z.title[z.date == d]:
        atbilde.append(title)
        
    df.loc[len(df)] = atbilde

In [ ]:
df.to_sql('main', connection, if_exists='append',index=False)

In [ ]:
atbilde = []
atbilde.append(z.date[0])
#print(atbilde)

for title in z.title:
    atbilde.append(title)
        
print(atbilde)
df.loc[len(df)] = atbilde

# delete

connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("DELETE from top30 where moment  between '2021-03-30 11:17:11.915369' and '2021-03-30 11:18:48.647641'")
cursor.execute(sql)
connection.commit()

# Yahoo finance

In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
df = pd.DataFrame(columns = ['date','SP500_Open', 'SP500_High', 'SP500_Low', 'SP500_Close', 'SP500_Adj_Close', 'SP500_Volume'])
df

In [ ]:
str([str('SP500_')+i for i in df.columns])

## S&P 500 (^GSPC)
SNP - SNP Real Time Price. Currency in USD

In [3]:
url = "https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC"

In [4]:
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

In [5]:
answer = []
for i in soup.find_all('td')[0:7]:
    #print(i.span)
    print(i.span.text)
    answer.append(i.span.text)
    #print(len(answer)    

May 07, 2021
4,210.34
4,238.04
4,201.64
4,232.60
4,232.60
4,013,060,000


for i in range(0, int(len(answer))//7):
    atbilde = []
    #print(i)
    for j in range (0,7):
        if j == 0:
            atbilde.append(pd.to_datetime(answer[i*7+j], infer_datetime_format=True))
        else:
            atbilde.append(answer[i*7+j])
    df.loc[len(df)] = atbilde

In [63]:
if str(pd.to_datetime(soup.find_all('td')[0].text, infer_datetime_format=True)) == '2021-05-06 00:00:00':
    print('Stop. This data point is already in the data base')
else:
    print('Lets get scraping')
    answer = []
    answer.append(str(pd.to_datetime(soup.find_all('td')[0].text, infer_datetime_format=True)))
    
    #for i in soup.find_all('td')[1:7]:
        #answer.append(i.span.text)
    answer = answer + [i.span.text for i in soup.find_all('td')[1:7]]
        
answer

Lets get scraping


['2021-05-07 00:00:00',
 '4,210.34',
 '4,238.04',
 '4,201.64',
 '4,232.60',
 '4,232.60',
 '4,013,060,000']

In [54]:
str(pd.to_datetime(soup.find_all('td')[0].text, infer_datetime_format=True)) == '2021-05-07 00:00:00'

True

In [7]:
answer

['May 07, 2021',
 '4,210.34',
 '4,238.04',
 '4,201.64',
 '4,232.60',
 '4,232.60',
 '4,013,060,000']

In [ ]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("Select max(date) as date from SP500")
z = pd.read_sql_query(sql,connection)

yahoo = str(pd.to_datetime(answer[0], infer_datetime_format=True))
db = z.date

print(yahoo == db)
    #print('Not yet')

In [ ]:
answer = []
for i in soup.find_all('td')[0:7]:
    print(i.span.text)
    answer.append(i.span.text)

In [ ]:
df.to_sql('SP500', connection, if_exists='append',index=False)

## Dow Jones Industrial Average (^DJI)
DJI - DJI Real Time Price. Currency in USD

In [ ]:
url = "https://finance.yahoo.com/quote/%5EDJI/history?p=%5EDJI"

In [ ]:
df = pd.DataFrame(columns = ['date', 'DJI_Open', 'DJI_High', 'DJI_Low', 'DJI_Close', 'DJI_Adj_Close', 'DJI_Volume'])
df

In [ ]:
columns_new = [str('DJI_')+i for i in df.columns]

In [ ]:
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

In [ ]:
answer = []
for i in soup.find_all('td'):
    #print(i.span)
    #print(i.span.text)
    answer.append(i.span.text)
    #print(len(answer)  

In [ ]:
for i in range(0, int(len(answer))//7):
    atbilde = []
    #print(i)
    for j in range (0,7):
        if j == 0:
            atbilde.append(pd.to_datetime(answer[i*7+j], infer_datetime_format=True))
        else:
            atbilde.append(answer[i*7+j])
    df.loc[len(df)] = atbilde

In [ ]:
df

In [ ]:
df.to_sql('DJI', connection, if_exists='append',index=False)

# Merge

In [ ]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()

sql = ('''Select 
            main.date,
            top1,top2,top3,top4,top5,
            top6,top7,top8,top9,top10,
            top11,top12,top13,top14,top15,
            top16,top17,top18,top19,top20,
            top21,top22,top23,top24,top25,
            top26,top27,top28,top29,top30,
            
            DJI_Open,
            DJI_High,
            DJI_Low,
            DJI_Close,
            DJI_Adj_Close,
            
            SP500_Open,
            SP500_High,
            SP500_Low,
            SP500_Close,
            SP500_Adj_Close

            from main
            left join DJI on
            main.date = date(DJI.Date)
            left join SP500 on
            main.date = date(SP500.Date)

       ''')
z = pd.read_sql_query(sql,connection)
z

In [ ]:
z.isnull().sum()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob